<a href="https://colab.research.google.com/github/JoeJiang3936/NLP/blob/master/Bert_SQuAD1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 38.3MB/s 
     |████████████████████████████████| 860kB 42.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=45e25e7d19de18d76d74ca9e2da1dce816a448d50e4b523a7483bdfb1995aa75
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [2]:
!pip install transformers

     |████████████████████████████████| 450kB 2.7MB/s 


In [3]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=dc547d6193932d41d19e2aa432b2693291fadf2495c145e22942f4a3cf7062a9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [4]:
import wget
wget.download("https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")
wget.download("https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json")
wget.download("https://raw.githubusercontent.com/allenai/bi-att-flow/master/squad/evaluate-v1.1.py")
wget.download("https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt")
wget.download('https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/utils_squad.py')
wget.download('https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/utils_squad_evaluate.py')

'utils_squad_evaluate.py'

In [5]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [6]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("There are %d GPU(s) available." % torch.cuda.device_count())
  print("We will use GPU:", torch.cuda.get_device_name(0))
else:
  print('There are no GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use GPU: Tesla P100-PCIE-16GB


In [0]:
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from pytorch_transformers import BertForQuestionAnswering, BertTokenizer
from utils_squad import read_squad_examples, convert_examples_to_features, RawResult, write_predictions

In [0]:
epochs = 2
batch_size = 16

In [0]:
import numpy as np
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42) 

In [10]:
#initialize Bert base model 
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

100%|██████████| 440473133/440473133 [00:37<00:00, 11610054.18B/s]


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [0]:
train_file = './train-v1.1.json'
train_examples = read_squad_examples(train_file, is_training = True, version_2_with_negative=False)

In [12]:
len(train_examples)

87599

In [13]:
train_examples[0]

qas_id: 5733be284776f41900661182, question_text: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?, doc_tokens: [Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.], start_position: 90, end_position: 92

In [0]:
#tokenize the training examples
tokenizer = BertTokenizer(vocab_file='bert-base-uncased-vocab.txt')
train_features = convert_examples_to_features(train_examples, tokenizer, max_seq_length=384, doc_stride=128, max_query_length=64, is_training=True)

In [15]:
len(train_features)

88641

In [0]:
# convert the tokenized data into torch tensors
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype = torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype = torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype = torch.long)
all_start_positions = torch.tensor([f.start_position for f in train_features], dtype = torch.long)
all_end_positions = torch.tensor([f.end_position for f in train_features], dtype = torch.long)

In [17]:
all_input_ids[1], all_input_mask[1], all_segment_ids[1], all_start_positions[1], all_end_positions[1]

(tensor([  101,  2054,  2003,  1999,  2392,  1997,  1996, 10289,  8214,  2364,
          2311,  1029,   102,  6549,  2135,  1010,  1996,  2082,  2038,  1037,
          3234,  2839,  1012, 10234,  1996,  2364,  2311,  1005,  1055,  2751,
          8514,  2003,  1037,  3585,  6231,  1997,  1996,  6261,  2984,  1012,
          3202,  1999,  2392,  1997,  1996,  2364,  2311,  1998,  5307,  2009,
          1010,  2003,  1037,  6967,  6231,  1997,  4828,  2007,  2608,  2039,
         14995,  6924,  2007,  1996,  5722,  1000,  2310,  3490,  2618,  4748,
          2033, 18168,  5267,  1000,  1012,  2279,  2000,  1996,  2364,  2311,
          2003,  1996, 13546,  1997,  1996,  6730,  2540,  1012,  3202,  2369,
          1996, 13546,  2003,  1996, 24665, 23052,  1010,  1037, 14042,  2173,
          1997,  7083,  1998,  9185,  1012,  2009,  2003,  1037, 15059,  1997,
          1996, 24665, 23052,  2012, 10223, 26371,  1010,  2605,  2073,  1996,
          6261,  2984, 22353,  2135,  2596,  2000,  

In [0]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_start_positions, all_end_positions)

In [19]:
train_data[1]

(tensor([  101,  2054,  2003,  1999,  2392,  1997,  1996, 10289,  8214,  2364,
          2311,  1029,   102,  6549,  2135,  1010,  1996,  2082,  2038,  1037,
          3234,  2839,  1012, 10234,  1996,  2364,  2311,  1005,  1055,  2751,
          8514,  2003,  1037,  3585,  6231,  1997,  1996,  6261,  2984,  1012,
          3202,  1999,  2392,  1997,  1996,  2364,  2311,  1998,  5307,  2009,
          1010,  2003,  1037,  6967,  6231,  1997,  4828,  2007,  2608,  2039,
         14995,  6924,  2007,  1996,  5722,  1000,  2310,  3490,  2618,  4748,
          2033, 18168,  5267,  1000,  1012,  2279,  2000,  1996,  2364,  2311,
          2003,  1996, 13546,  1997,  1996,  6730,  2540,  1012,  3202,  2369,
          1996, 13546,  2003,  1996, 24665, 23052,  1010,  1037, 14042,  2173,
          1997,  7083,  1998,  9185,  1012,  2009,  2003,  1037, 15059,  1997,
          1996, 24665, 23052,  2012, 10223, 26371,  1010,  2605,  2073,  1996,
          6261,  2984, 22353,  2135,  2596,  2000,  

In [20]:
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
len(train_dataloader)

5541

In [21]:
dev_file = './dev-v1.1.json'
dev_examples = read_squad_examples(dev_file, is_training = False, version_2_with_negative=False)

print("total evaluation samples = ", len(dev_examples))
dev_examples[0]

total evaluation samples =  10570


qas_id: 56be4db0acb8001400a502ec, question_text: Which NFL team represented the AFC at Super Bowl 50?, doc_tokens: [Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.]

In [22]:
dev_features = convert_examples_to_features(dev_examples, tokenizer, max_seq_length=384, doc_stride=128, max_query_length=64, is_training=False)
len(dev_features)

10833

In [23]:
# convert dev tokenized data into torch tensors
all_input_ids = torch.tensor([f.input_ids for f in dev_features], dtype = torch.long)
all_input_mask = torch.tensor([f.input_mask for f in dev_features], dtype = torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in dev_features], dtype = torch.long)
all_example_index = torch.arange(all_input_ids.size(0), dtype = torch.long)

all_input_ids[1], all_input_mask[1], all_segment_ids[1], all_example_index[1]

(tensor([  101,  2029,  5088,  2136,  3421,  1996, 22309,  2012,  3565,  4605,
          2753,  1029,   102,  3565,  4605,  2753,  2001,  2019,  2137,  2374,
          2208,  2000,  5646,  1996,  3410,  1997,  1996,  2120,  2374,  2223,
          1006,  5088,  1007,  2005,  1996,  2325,  2161,  1012,  1996,  2137,
          2374,  3034,  1006, 10511,  1007,  3410,  7573, 14169,  3249,  1996,
          2120,  2374,  3034,  1006, 22309,  1007,  3410,  3792, 12915,  2484,
          1516,  2184,  2000,  7796,  2037,  2353,  3565,  4605,  2516,  1012,
          1996,  2208,  2001,  2209,  2006,  2337,  1021,  1010,  2355,  1010,
          2012, 11902,  1005,  1055,  3346,  1999,  1996,  2624,  3799,  3016,
          2181,  2012,  4203, 10254,  1010,  2662,  1012,  2004,  2023,  2001,
          1996, 12951,  3565,  4605,  1010,  1996,  2223, 13155,  1996,  1000,
          3585,  5315,  1000,  2007,  2536,  2751,  1011, 11773, 11107,  1010,
          2004,  2092,  2004,  8184, 28324,  2075,  

In [24]:
dev_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_example_index)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)
len(dev_dataloader)

678

In [0]:
#set up the optimizer
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(), lr = 3e-5, eps = 1e-4, correct_bias=False)
total_steps = len(train_dataloader)*epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [27]:
import time 

print('Training the model ...')
time0 = time.time()
lossList = []
for epoch in range(4):  
  total_loss = 0
  for step, batch in enumerate(train_dataloader):
    model.train()
    if step%40 == 0 and not step == 0:
      elapsed = time.time() - time0
      print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
   
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, segment_ids, start_positions, end_positions = batch
    outputs = model(input_ids, segment_ids, input_mask, start_positions, end_positions)
    loss = outputs[0]
    total_loss += loss
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    model.zero_grad()
  avg_train_loss = (total_loss/len(train_dataloader)).detach().cpu().numpy()
  lossList.append(avg_train_loss)
  print('training time for epoch: ', epoch, ': ', time.time()-time0)
  print('Average loss after epoch ', epoch, ': ', avg_train_loss)


Training the model ...
  Batch    40  of  5,541.    Elapsed: 27.935028314590454.
  Batch    80  of  5,541.    Elapsed: 55.86741089820862.
  Batch   120  of  5,541.    Elapsed: 83.76790237426758.
  Batch   160  of  5,541.    Elapsed: 111.66671466827393.
  Batch   200  of  5,541.    Elapsed: 139.54198908805847.
  Batch   240  of  5,541.    Elapsed: 167.45866084098816.
  Batch   280  of  5,541.    Elapsed: 195.34645342826843.
  Batch   320  of  5,541.    Elapsed: 223.24408769607544.
  Batch   360  of  5,541.    Elapsed: 251.1399393081665.
  Batch   400  of  5,541.    Elapsed: 278.9976963996887.
  Batch   440  of  5,541.    Elapsed: 306.89480996131897.
  Batch   480  of  5,541.    Elapsed: 334.76584935188293.
  Batch   520  of  5,541.    Elapsed: 362.68334102630615.
  Batch   560  of  5,541.    Elapsed: 390.5684218406677.
  Batch   600  of  5,541.    Elapsed: 418.4500529766083.
  Batch   640  of  5,541.    Elapsed: 446.3156006336212.
  Batch   680  of  5,541.    Elapsed: 474.19629192352295

In [0]:
# Save the trained model (Create the directory if it does not exist; otherwise override the contents)
import os
if not os.path.exists('./saved_model'):
    os.makedirs('./saved_model')

model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained('./saved_model')

In [66]:
#Validation of the model on dev dataset 

print("Validating the model...")
model.eval()
results = []
for input_ids, input_masks, segment_ids, example_indices in dev_dataloader:
  input_ids = input_ids.to(device)
  input_masks = input_masks.to(device)    
  segment_ids = segment_ids.to(device)    
  with torch.no_grad(): 
    batch_start_logits, batch_end_logits = model(input_ids, segment_ids, input_masks)
  for i, example_index in enumerate(example_indices):
    start_logits = batch_start_logits[i].detach().cpu().tolist()
    end_logits = batch_end_logits[i].detach().cpu().tolist()
    dev_feature = dev_features[example_index.item()]
    unique_id = int(dev_feature.unique_id)
    rawResult = RawResult(unique_id=unique_id, start_logits=start_logits, end_logits=end_logits)
    results.append(rawResult)
        
if not os.path.exists('./model_eval'):
    os.makedirs('./model_eval')
output_prediction_file = os.path.join("./model_eval", "predictions.json")
output_nbest_file = os.path.join("/model_eval", "nbest_predictions.json")
output_null_log_odds_file = os.path.join("./model_eval", "null_odds.json")

preds = write_predictions(dev_examples, dev_features, results, 20, 30, True, output_prediction_file, output_nbest_file, output_null_log_odds_file, True, False, 0.0)     

Validating the model...


FileNotFoundError: ignored

In [63]:
%run evaluate-v1.1.py dev-v1.1.json ./model_eval/predictions.json

{"exact_match": 79.81078524124882, "f1": 87.50959699791207}
